In [1]:
import os
from experiment_setup import setups
from model_tracker import track_training_C02_emissions

In [3]:
# download the dataset, the links can be fund in the README
dataset = "rees46"
dataset_path = f"../datasets/{dataset}"
model_path = "../trained_models"

## Run the preprocess script, specific to the dataset you chose

- The preprocessing script in general, executes the following steps:
    - Loads the raw data, with correct types
    - Creates the sessions
    - Removes duplicated items. An item is considered as a duplicate if the preceding (based on time) event in the same session contains the exact same item.
    - Performes iterative support filtering
        - Removes sessions with only one event
        - Removes items with less than 5 events
        - Until the size of the dataset changes


In [ ]:
%run coveo_preproc.py --path $dataset_path

## Use a specific setup for your dataset

In [4]:
params = setups[dataset]["params_bprmax"]

In [5]:
train_path = os.path.join(dataset_path,f"{dataset}_processed_view_train_full.tsv")
test_path = os.path.join(dataset_path,f"{dataset}_processed_view_test.tsv")

In [6]:
def create_recpack_script(model_name, train_path, test_path, loss, optim, layers, embedding, hidden_size, batch_size, dropout_p_embed, dropout_p_hidden, learning_rate, bpreg, n_epochs, n_sample, m, use_correct_weight_init):
    s_train_test_full = (
        f"python ../recpack/main.py --train_path {train_path} "
        f"--test_path {test_path} --m {m} --n_epochs {n_epochs} --loss {loss} --optim {optim} "
        f"--layers {layers} --embedding {embedding} --hidden_size {hidden_size} "
        f"--batch_size {batch_size} --dropout_p_embed {dropout_p_embed} "
        f"--dropout_p_hidden {dropout_p_hidden} --learning_rate {learning_rate}"
        f"{f' --n_sample {n_sample}' if loss=='bpr-max' else ''}"
        f"{' --use_correct_weight_init' if use_correct_weight_init else ''}"
        f"{f' --bpreg {bpreg}' if loss=='bpr-max' else ''}"
    )
    return s_train_test_full

In [7]:
loss = params["loss"]
optim = params["optim"]
const_emb = params["constrained_embedding"]
embed = params["embedding"]
final_act = params["final_act"]
layers = params["layers"]
batch_size = params["batch_size"]
dropout_p_embed = params["dropout_p_embed"]
dropout_p_hidden = params["dropout_p_hidden"]
learning_rate = params["learning_rate"]
momentum = params["momentum"]
sample_alpha = params["sample_alpha"]
bpreg = params["bpreg"]
logq = params["logq"]
hidden_act = params["hidden_act"]
n_sample = params["n_sample"]
n_epochs = 5
m = '1 5 10 20'

## Train & test (major fix model)

In [8]:
train_test_script_majorfix = create_recpack_script(model_name='recpack_majorfix_bprmax', train_path=train_path, test_path=test_path, loss=loss, optim=optim, layers=1, embedding=layers, hidden_size=layers, batch_size=batch_size, dropout_p_embed=dropout_p_embed, dropout_p_hidden=dropout_p_hidden, learning_rate=learning_rate, bpreg=bpreg, n_epochs=n_epochs, n_sample=n_sample, m=m, use_correct_weight_init=True)

In [9]:
track_training_C02_emissions(train_test_script_majorfix, "recpack_majorfix_bprmax", dataset)

[codecarbon WARNING @ 16:57:42] Invalid gpu_ids format. Expected a string or a list of ints.
[codecarbon INFO @ 16:57:42] [setup] RAM Tracking...
[codecarbon INFO @ 16:57:42] [setup] GPU Tracking...
[codecarbon INFO @ 16:57:43] Tracking Nvidia GPU via pynvml
[codecarbon INFO @ 16:57:43] [setup] CPU Tracking...
[codecarbon WARNING @ 16:57:43] No CPU tracking mode found. Falling back on CPU constant mode.
[codecarbon WARNING @ 16:57:45] We saw that you have a 13th Gen Intel(R) Core(TM) i9-13900HX but we don't know it. Please contact us.
[codecarbon INFO @ 16:57:45] CPU Model on constant consumption mode: 13th Gen Intel(R) Core(TM) i9-13900HX
[codecarbon WARNING @ 16:57:46] Failed to retrieve gpu information
Traceback (most recent call last):
  File "c:\Users\EVILAB\AppData\Local\Programs\Python\Python312\Lib\site-packages\codecarbon\core\gpu.py", line 238, in get_gpu_details
    devices_info.append(gpu_device.get_gpu_details())
                        ^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File 